In [1]:
import json
import pandas as pd
from typing import List, Dict
from itertools import groupby
import math

with open("ward_alangulam.json", "r", encoding="utf-8") as f:
    res = json.load(f)
    idx_data = res["index_data"]

df = pd.DataFrame(idx_data)


class COMPARE_TYPE:
    AC = "ac"
    LOCAL_BODY = "local_body"
    BOOTH = "booth"
    LOCALITY = "locality"
    WARD = "ward_vp"


In [10]:
def add_index_row_style(
    source_row_data: List, should_add_booths_per_lb: bool, compare_type
):
    modified_row_data: List = []

    if compare_type in [COMPARE_TYPE.LOCALITY, COMPARE_TYPE.WARD]:
        sub_group_key = compare_type # compare_type would be "locality" or "ward_vp"
        
        df = pd.DataFrame(source_row_data)
        lb_list = df["lb_name"].unique().tolist()
        
        ## set sl no based on lb names
        lb_dict_with_sl_no = { lb_name : idx+1 for idx, lb_name in enumerate(lb_list) }
        for index_item in source_row_data:
            index_item["s_no"] = lb_dict_with_sl_no[index_item["lb_name"]]

        ## lb wise total booth count
        for lb_name in lb_list:
            matching_records_df = df[(df["lb_name"] == lb_name)]
            total = matching_records_df["booth_count"].sum()
            index_list = matching_records_df.index.tolist()

            for idx in index_list:
                source_row_data[idx]["lb_wise_booth_count"] = int(total)

        ## ward_vp/locality wise total booth count
        ward_list = df[["lb_name", sub_group_key]]
        new_ward_list = set(ward_list.to_records(index=None).tolist())
        
        for lb_name, ward_vp in new_ward_list:
            matching_records_df = df[(df["lb_name"] == lb_name) & (df[sub_group_key] == ward_vp)]
            total = matching_records_df["booth_count"].sum()
            index_list = matching_records_df.index.tolist()

            for idx in index_list:
                source_row_data[idx]["ward_wise_booth_count"] = int(total)
        
        ## trend wise total booth count
        trend_list_mapping = df[["lb_name", "ward_vp", "trend"]]
        new_trend_list_mapping = set(trend_list_mapping.to_records(index=None).tolist())

        for lb_name, ward_vp, trend in new_trend_list_mapping:
            matching_records_df = df[(df["lb_name"] == lb_name) & (df["ward_vp"] == ward_vp) & (df["trend"] == trend)]
            total = matching_records_df["booth_count"].sum()
            index_list = matching_records_df.index.tolist()

            for idx in index_list:
                source_row_data[idx]["trend_wise_booth_count"] = int(total)

        # pd.DataFrame(source_row_data)
        return source_row_data
        
    else:
        is_dark_theme: bool = True

        grouped_data = groupby(source_row_data, lambda x: x["lb_name"])

        current_trend = None
        current_parent_name = None

        for key, value in grouped_data:
            is_dark_theme = not is_dark_theme

            list_value = list(value)
            length = len(list_value)

            total_booth_count = 0
            for idx, item in enumerate(list_value):
                if idx == 0:
                    item["add_thick_border"] = True

                if current_trend == None and current_parent_name == None:
                    item["has_top_line"] = True
                    current_trend = item["trend"]
                    current_parent_name = item["lb_name"]
                elif (
                    item["lb_name"] == current_parent_name
                    and item["trend"] == current_trend
                ):
                    item["has_top_line"] = False
                else:
                    item["has_top_line"] = True
                    current_trend = item["trend"]
                    current_parent_name = item["lb_name"]

                item["row_theme"] = is_dark_theme

                if should_add_booths_per_lb:
                    total_booth_count += item["booth_count"]

                # if last item, add the total booth count value for it to be displayed in the index data
                if idx + 1 == length:
                    item["total_booth_count"] = total_booth_count

                modified_row_data.append(item)

        # get total booth count per local body and trend
        current_lb = None
        current_trend = None
        first_trendwise_idx = None
        booth_count_per_trend = 0

        total_length = len(modified_row_data)

        for current_idx, current_row in enumerate(modified_row_data):
            # initial set up even before the first row is iterated
            if (current_lb and current_trend and first_trendwise_idx) == None:
                current_lb = current_row["lb_name"]
                current_trend = current_row["trend"]
                first_trendwise_idx = current_idx

            # if lb and trend match, get the booth count from the row and add to the total booth count
            belongs_to_lb_trend_bucket = (
                current_lb == current_row["lb_name"]
                and current_trend == current_row["trend"]
            )

            if belongs_to_lb_trend_bucket:
                booth_count_per_trend += current_row["booth_count"]

                # if row is the last item and doesnt match the previous bucket
                if total_length == current_idx + 1:
                    prev_first_row = modified_row_data[first_trendwise_idx]
                    prev_first_row["total_booth_count_per_trend"] = (
                        booth_count_per_trend
                    )
            else:
                prev_first_row = modified_row_data[first_trendwise_idx]
                prev_first_row["total_booth_count_per_trend"] = booth_count_per_trend

                booth_count_per_trend = current_row["booth_count"]
                current_lb = current_row["lb_name"]
                current_trend = current_row["trend"]
                first_trendwise_idx = current_idx

                # if row is the last item and doesnt match the previous bucket
                if total_length == current_idx + 1:
                    current_row["total_booth_count_per_trend"] = booth_count_per_trend

    #
    #
    #

    return modified_row_data

def add_other_table_row_style(source_row_data: List, should_add_booths_per_lb: bool):
    is_dark_theme: bool = True
    modified_row_data: List = []

    grouped_data = groupby(source_row_data, lambda x: x["lb_name"])

    for key, value in grouped_data:
        is_dark_theme = not is_dark_theme

        list_value = list(value)
        length = len(list_value)

        total_booth_count = 0
        for idx, item in enumerate(list_value):
            item["row_theme"] = is_dark_theme

            if should_add_booths_per_lb:
                total_booth_count += item["booth_count"]

            # if last item, add the total booth count value for it to be displayed in the index data
            if idx + 1 == length:
                item["total_booth_count"] = total_booth_count

            modified_row_data.append(item)

    return modified_row_data

def merge_index_table_rows(index_table_data: list, compare_type) -> List:
    grouped_index_table_data = []

    if compare_type in [COMPARE_TYPE.WARD, COMPARE_TYPE.LOCALITY]:
        for index_page_obj in index_table_data:
            new_page_data = dict() 
            for index_item in index_page_obj["page_data"]["page_data"]:

                new_page_data = {
                    **new_page_data,
                    index_item["lb_name"]: {
                        "meta": {
                            "lb_type": index_item["lb_type"],
                            "total_booths": index_item["lb_wise_booth_count"],
                        },
                        "data": {
                            **new_page_data.get(index_item["lb_name"], {}).get("data", {}),
                            index_item[compare_type]: {
                                "meta": {
                                    "total_booths": index_item["ward_wise_booth_count"],
                                    "ward_type": index_item["ward_vp_type"],
                                },
                                "data": {
                                    **new_page_data.get(index_item["lb_name"], {}).get("data", {}).get(index_item[compare_type], {}).get("data", {}),
                                    index_item["trend"]: {
                                        "meta": {
                                            "total_booths": index_item["trend_wise_booth_count"],
                                        },
                                        "data": [
                                            *new_page_data.get(index_item["lb_name"], {}).get("data", {}).get(index_item[compare_type], {}).get("data", {}).get(index_item["trend"], {}).get("data", []),
                                            index_item,
                                        ]
                                    }
                                }
                            }
                        }
                    }
                }


            index_page_obj["page_data"]["grouped_page_data"] = new_page_data
        
        return index_table_data
    else:
        return index_table_data

In [13]:
# add_index_row_style(idx_data, True, COMPARE_TYPE.WARD)

page_data = add_index_row_style(idx_data, True, COMPARE_TYPE.WARD)
pd.DataFrame(page_data).head(80)

# with open("new.json", "w", encoding="utf-8") as f:
#     f.write(json.dumps(page_data, ensure_ascii=False))

,lb_name,lb_type,ward_vp,ward_vp_type,booth_list,booth_list_expanded,booth_count,trend,sub_trend,page_no,s_no,page_no_list,lb_wise_booth_count,ward_wise_booth_count,trend_wise_booth_count
0,ஆலங்குளம் தெற்கு,ஒன்றியம்,மாயமான்குரிச்சி,vp,[78],[78],1,WW,SS,2,1,3 - 4,56,8,1
1,ஆலங்குளம் தெற்கு,ஒன்றியம்,மாயமான்குரிச்சி,vp,"[76, 81-85]","[76, 81, 82, 83, 84, 85]",6,WL,SD,2,1,5 - 8,56,8,6
2,ஆலங்குளம் தெற்கு,ஒன்றியம்,மாயமான்குரிச்சி,vp,[77],[77],1,LL,DD,2,1,9 - 10,56,8,1
3,ஆலங்குளம் தெற்கு,ஒன்றியம்,நாரணபுரம்,vp,"[79,80, 86]","[79, 80, 86]",3,WW,SS,2,1,11 - 13,56,3,3
4,ஆலங்குளம் தெற்கு,ஒன்றியம்,கடங்கநேரி,vp,[107],[107],1,WW,SS,2,1,14 - 15,56,8,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,கடையம் தெற்கு,ஒன்றியம்,பொட்டல்புதூர்,vp,[255],[255],1,LW,DS,9,4,168 - 169,46,7,1
76,கடையம் தெற்கு,ஒன்றியம்,பொட்டல்புதூர்,vp,[253],[253],1,LL,DB,9,4,170 - 171,46,7,1
77,கடையம் தெற்கு,ஒன்றியம்,பாப்பான்குளம்,vp,[258],[258],1,WW,SS,9,4,172 - 173,46,5,1
78,கடையம் தெற்கு,ஒன்றியம்,பாப்பான்குளம்,vp,"[257, 259-261]","[257, 259, 260, 261]",4,WL,SD,9,4,174 - 176,46,5,4


In [80]:
def get_page_obj(pg_no, page_data) -> Dict:
    return {"page_number": str(pg_no), "page_data": page_data}


def get_total_hierarchy_item_count(data_to_loop: List, KEY_TO_LOOKUP: str):
    count = 0
    for item in data_to_loop:
        count += item[KEY_TO_LOOKUP]
    return count


def group_table_pages_data(
    response_data: Dict,
    ITEMS_PER_PAGE: int,
    should_set_row_style: bool,
    should_add_booths_per_lb: bool,
    is_get_total_hierarchy_item_count: bool,
    TABLE_KEY_TO_LOOKUP: str,
    should_sort: bool,
    is_index_table: bool,
    compare_type: str,
):
    if len(response_data) == 0:
        return []

    if should_sort:
        response_data = sorted(response_data, key=lambda x: x["booth_no"])
    
    ## set alternating row style based on the local body name
    if should_set_row_style:
        if is_index_table:
            page_data = add_index_row_style(response_data, should_add_booths_per_lb, compare_type)
        else:
            page_data = add_other_table_row_style(
                response_data, should_add_booths_per_lb
            )
    else:
        page_data = response_data
    page_item_count = len(page_data)
    final_page_data = []

    if page_item_count < ITEMS_PER_PAGE:
        final_page_data.append(
            get_page_obj(
                "",
                {
                    "total_count_per_table": (
                        get_total_hierarchy_item_count(
                            data_to_loop=page_data, KEY_TO_LOOKUP=TABLE_KEY_TO_LOOKUP
                        )
                        if is_get_total_hierarchy_item_count
                        else None
                    ),
                    "page_data": page_data,
                },
            )
        )
    else:
        number_of_iterations = math.ceil(page_item_count / ITEMS_PER_PAGE)
        slice_from = 0
        slice_till = 0 + ITEMS_PER_PAGE

        for x in range(number_of_iterations):
            sliced_page_data = page_data[slice_from:slice_till]

            final_page_data.append(
                get_page_obj(
                    "",
                    {
                        "total_count_per_table": (
                            get_total_hierarchy_item_count(
                                data_to_loop=sliced_page_data,
                                KEY_TO_LOOKUP=TABLE_KEY_TO_LOOKUP,
                            )
                            if is_get_total_hierarchy_item_count
                            else None
                        ),
                        "page_data": sliced_page_data,
                    },
                )
            )
            slice_from = slice_from + ITEMS_PER_PAGE
            slice_till = slice_till + ITEMS_PER_PAGE
    
    
    # does grouping of the index data items in order to group and merge the rows as per lb, locality/ward_vp wise
    # should do only for the index table, not for summary or appendix table
    if is_index_table and should_set_row_style:
        final_page_data = merge_index_table_rows(final_page_data, compare_type)

    return final_page_data


index_data_pages = group_table_pages_data(
    response_data=idx_data,
    ITEMS_PER_PAGE=15,
    should_set_row_style=True,
    should_add_booths_per_lb=True,
    is_get_total_hierarchy_item_count=True,
    TABLE_KEY_TO_LOOKUP="booth_count",
    should_sort=False,
    is_index_table=True,
    compare_type=COMPARE_TYPE.WARD,
)

# pd.DataFrame(index_data_pages)
index_data_pages

[{'page_number': '',
  'page_data': {'total_count_per_table': 29,
   'page_data': [{'lb_name': 'ஆலங்குளம் தெற்கு',
     'lb_type': 'ஒன்றியம்',
     'ward_vp': 'மாயமான்குரிச்சி',
     'ward_vp_type': 'vp',
     'booth_list': ['78'],
     'booth_list_expanded': [78],
     'booth_count': 1,
     'trend': 'WW',
     'sub_trend': 'SS',
     'page_no': 2,
     's_no': 1,
     'page_no_list': '3 - 4',
     'ward_wise_booth_count': 8,
     'lb_wise_booth_count': 56,
     'trend_wise_booth_count': 1},
    {'lb_name': 'ஆலங்குளம் தெற்கு',
     'lb_type': 'ஒன்றியம்',
     'ward_vp': 'மாயமான்குரிச்சி',
     'ward_vp_type': 'vp',
     'booth_list': ['76', '81-85'],
     'booth_list_expanded': [76, 81, 82, 83, 84, 85],
     'booth_count': 6,
     'trend': 'WL',
     'sub_trend': 'SD',
     'page_no': 2,
     's_no': 2,
     'page_no_list': '5 - 8',
     'ward_wise_booth_count': 8,
     'lb_wise_booth_count': 56,
     'trend_wise_booth_count': 6},
    {'lb_name': 'ஆலங்குளம் தெற்கு',
     'lb_type': 'ஒ

In [81]:
with open("new2.json", "w", encoding="utf-8") as f:
    f.write(json.dumps(index_data_pages, ensure_ascii=False))

In [9]:
test_list = ["test1", "test2", "test3"]

test_dict = {test: idx+1 for idx, test in enumerate(test_list)}

type(test_dict["test1"])

int

In [6]:
x_axis = ["AIADMK", "DMK", "IND 3", "NTK", "Others", "CPI(M)"]
PARTY_WISE_COLORS = {
    "DMK": "#e82b1e",
    "VCK": "#0961b3",
    "BJP": "#e87007",
    "AIADMK": "#215e1b",
    "PMK": "#d9ce09",
    "NTK": "#141210",
    "Others": "#b3b4b5",
    "IND" : "#4c516d",
    "MDMK" :"#950606",
    "MNM" : "#d6204e",
    "IND" : "#4c516d",
    "PT"  :  "#66bd63",
    "CPI" : "#f04545",
    "CPIM" : "#ad4040",
    "INC" : "#5fb2d9",
    "IUML" : "#19d776",
    "DMDK":"#FDDA0D",
    "AMMK": "#234735"
}

dmk_red_color = "#f04545"

for item in x_axis:
    print(len(item.split(" ")))

1
1
2
1
1
1


In [12]:
((308 - 308) != 0)

False

In [17]:
import json 
with open("ac_lb_mapping.json", "r", encoding="utf-8") as f:
    ac_lb_map = json.load(f)